## Round-Robin Group Chat
`RoundRobinGroupChat` is a simple team that allows all agents to share context and take turns to respond in a round-robin fashion. 

On its turn, each agent broadcasts its response to all other agents in the team, so **all agents have the same context**.

We will start by creating a team with a single AssistantAgent agent and TextMentionTermination termination condition that stops the team when a word is detected.

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

print(os.getenv('CHAT_MODEL'))

gpt-4o-mini


In [2]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient


project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=os.environ["AIPROJECT_CONNECTION_STRING"]
)

base_url = project_client.inference.get_azure_openai_client(
    api_version="2024-06-01").base_url

api_endpoint = f'https://{base_url.host}/'

api_key = project_client.inference.get_azure_openai_client(
    api_version="2024-06-01").api_key

deployment_name = os.environ["CHAT_MODEL"]

aoai_client = AzureOpenAIChatCompletionClient(
    azure_endpoint=api_endpoint,
    model="gpt-4o-mini",
    azure_deployment=deployment_name,
    api_key=api_key,
    api_version="2024-06-01"
)

In [3]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import TextMentionTermination


# Define a tool that gets the weather for a city.
async def get_weather(city: str) -> str:
    """Get the weather for a city."""
    return f"The weather in {city} is 72 degrees and Sunny."


# Create an assistant agent.
weather_agent = AssistantAgent(
    "assistant",
    model_client=aoai_client,
    tools=[get_weather],
    system_message="Respond 'TERMINATE' when task is complete.",
)

# Define a termination condition.
text_termination = TextMentionTermination("TERMINATE")

# Create a single-agent team.
single_agent_team = RoundRobinGroupChat(
    [weather_agent], termination_condition=text_termination)

In [4]:
async def run_team() -> None:
    result = await single_agent_team.run(task="What is the weather in New York?")
    print(result)


# Use `asyncio.run(run_team())` when running in a script.
await run_team()

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='What is the weather in New York?', type='TextMessage'), ToolCallRequestEvent(source='assistant', models_usage=RequestUsage(prompt_tokens=70, completion_tokens=16), metadata={}, content=[FunctionCall(id='call_dWNDVkFkzK21SKBm8kecKdA9', arguments='{"city":"New York"}', name='get_weather')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='assistant', models_usage=None, metadata={}, content=[FunctionExecutionResult(content='The weather in New York is 72 degrees and Sunny.', name='get_weather', call_id='call_dWNDVkFkzK21SKBm8kecKdA9', is_error=False)], type='ToolCallExecutionEvent'), ToolCallSummaryMessage(source='assistant', models_usage=None, metadata={}, content='The weather in New York is 72 degrees and Sunny.', type='ToolCallSummaryMessage'), TextMessage(source='assistant', models_usage=RequestUsage(prompt_tokens=107, completion_tokens=14), metadata={}, content='The weather in New York

In [5]:
await single_agent_team.reset()  # Reset the team for the next run.

## Streaming Team Messages
Similar to agent’s `on_messages_stream()` method, you can stream the team’s messages by calling the `run_stream()` method. 

It will return a generator that yields the messages produced by the agents in the team as they are generated, and the last item will be the task result.

In [ ]:
from autogen_agentchat.base import TaskResult


async def run_team_stream() -> None:
    async for message in single_agent_team.run_stream(task="What is the weather in New York?"):
        if isinstance(message, TaskResult):
            print("Stop Reason:", message.stop_reason)
        else:
            print(message)


# Use `asyncio.run(run_team_stream())` when running in a script.
await run_team_stream()

source='user' models_usage=None metadata={} content='What is the weather in New York?' type='TextMessage'
source='assistant' models_usage=RequestUsage(prompt_tokens=70, completion_tokens=16) metadata={} content=[FunctionCall(id='call_FoExiAo7r57u5kZJgMFOuPks', arguments='{"city":"New York"}', name='get_weather')] type='ToolCallRequestEvent'
source='assistant' models_usage=None metadata={} content=[FunctionExecutionResult(content='The weather in New York is 72 degrees and Sunny.', name='get_weather', call_id='call_FoExiAo7r57u5kZJgMFOuPks', is_error=False)] type='ToolCallExecutionEvent'
source='assistant' models_usage=None metadata={} content='The weather in New York is 72 degrees and Sunny.' type='ToolCallSummaryMessage'
source='assistant' models_usage=RequestUsage(prompt_tokens=107, completion_tokens=14) metadata={} content='The weather in New York is 72 degrees and sunny.' type='TextMessage'
source='assistant' models_usage=RequestUsage(prompt_tokens=125, completion_tokens=5) metadata

In [ ]:
from autogen_agentchat.ui import Console

# Use `asyncio.run(single_agent_team.reset())` when running in a script.
await single_agent_team.reset()  # Reset the team for the next run.
# Use `asyncio.run(single_agent_team.run_stream(task="What is the weather in Seattle?"))` when running in a script.
await Console(
    single_agent_team.run_stream(task="What is the weather in Seattle?")
)  # Stream the messages to the console.

## Reflection Pattern
Now we will create a team with two agents that implements the Reflection pattern, which is a multi-agent design pattern that uses a critic agent to evaluate the responses of a primary agent.

See how the reflection pattern works using the Core API.

In this example, we will use the `AssistantAgent` agent class for both the primary and critic agents. We will use both the `TextMentionTermination` and `MaxMessageTermination` termination conditions together to stop the team.

In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination


# Create the primary agent.
primary_agent = AssistantAgent(
    "primary",
    model_client=aoai_client,
    system_message="You are a helpful AI assistant.",
)

# Create the critic agent.
critic_agent = AssistantAgent(
    "critic",
    model_client=aoai_client,
    system_message="Provide constructive feedback. Respond with 'APPROVE' to when your feedbacks are addressed.",
)

# Define a termination condition that stops the task if the critic approves.
text_termination = TextMentionTermination("APPROVE")
# Define a termination condition that stops the task after 5 messages.
max_message_termination = MaxMessageTermination(5)
# Combine the termination conditions using the `|`` operator so that the
# task stops when either condition is met.
termination = text_termination | max_message_termination

# Create a team with the primary and critic agents.
reflection_team = RoundRobinGroupChat(
    [primary_agent, critic_agent], termination_condition=termination)

In [ ]:
# Use `asyncio.run(Console(reflection_team.run_stream(task="Write a short poem about fall season.")))` when running in a script.
await Console(
    reflection_team.run_stream(task="Write a short poem about fall season.")
)

### Resuming Team
Let’s run the team again with a new task while keeping the context about the previous task.

In [ ]:
# Write the poem in Chinese Tang poetry style.
# Use `asyncio.run(Console(reflection_team.run_stream(task="Rewrite the poem in Korean")))` when running in a script.
await Console(reflection_team.run_stream(task="Rewrite the poem in Korean"))

In [ ]:
# Write the poem in Spanish.
# Use `asyncio.run(Console(reflection_team.run_stream(task="Write the poem in Chinese.")))` when running in a script.
await Console(reflection_team.run_stream(task="Write the poem in Chinese."))

### Resuming A Previous Task
We can call `run()` or `run_stream()` methods without setting the task again to resume the previous task. The team will continue from where it left off.

In [ ]:
# Use the `asyncio.run(Console(reflection_team.run_stream()))` when running in a script.
await Console(reflection_team.run_stream())

## Pause for User Input
Often times, team needs additional input from the application (i.e., user) to continue processing the task. We will show two possible ways to do it:

- Set the maximum number of turns such that the team stops after the specified number of turns.

- Use the `HandoffTermination` termination condition.

You can also use custom termination conditions, see Termination Conditions.

### Maximum Number of Turns
This is the simplest way to pause the team for user input. For example, you can set the maximum number of turns to 1 such that the team stops right after the first agent responds. This is useful when you want the user to constantly engage with the team, such as in a chatbot scenario.

Simply set the `max_turns` parameter in the `RoundRobinGroupChat()` constructor.

```python
team = RoundRobinGroupChat([...], max_turns=1)
```

Once the team stops, the turn count will be reset. When you resume the team, it will start from 0 again.

Note that `max_turn` is specific to the team class and is currently only supported by `RoundRobinGroupChat`, `SelectorGroupChat`, and `Swarm`. When used with termination conditions, the team will stop when either condition is met.

### Using Handoff to Pause Team
You can use the `HandoffTermination` termination condition to stop the team when an agent sends a `HandoffMessage` message.

Let’s create a team with a single `AssistantAgent` agent with a handoff setting.

In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.base import  Handoff
from autogen_agentchat.conditions import HandoffTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat


# Create a lazy assistant agent that always hands off to the user.
lazy_agent = AssistantAgent(
    "lazy_assistant",
    model_client=aoai_client,
    handoffs=[Handoff(target="user", message="Transfer to user.")],
    system_message="Always transfer to user when you don't know the answer. Respond 'TERMINATE' when task is complete.",
)

# Define a termination condition that checks for handoff message targetting helper and text "TERMINATE".
handoff_termination = HandoffTermination(target="user")
text_termination = TextMentionTermination("TERMINATE")
termination = handoff_termination | text_termination

# Create a single-agent team.
lazy_agent_team = RoundRobinGroupChat(
    [lazy_agent], termination_condition=termination)

In [ ]:
from autogen_agentchat.ui import Console

# Use `asyncio.run(Console(lazy_agent_team.run_stream(task="What is the weather in New York?")))` when running in a script.
await Console(lazy_agent_team.run_stream(task="What is the weather in New York?"))

In [ ]:
# Use `asyncio.run(Console(lazy_agent_team.run_stream(task="It is raining in New York.")))` when running in a script.
await Console(lazy_agent_team.run_stream(task="It is raining in New York."))